In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
# reading training set
df_train = pd.read_csv("/Users/fabbas1/Google Drive/study/Phd/Machine Learning/assignment/ITCS6156_SLProject/DigitRecognition/optdigits_raining.csv", 
                 header=None)
# reading test set
df_test = pd.read_csv("/Users/fabbas1/Google Drive/study/Phd/Machine Learning/assignment/ITCS6156_SLProject/DigitRecognition/optdigits_test.csv", 
                 header=None)

In [3]:
# split features from target values
X_train =  df_train.iloc[:, :64]
y_train = df_train.iloc[: , 64:]
print(y_train.shape)
print(X_train.shape)

(3823, 1)
(3823, 64)


In [4]:
# splite testing data 
X_test = df_test.iloc[:, :64]
y_test = df_test.iloc[: , 64:]

In [7]:
# actual decision tree (base case)
dt = DecisionTreeClassifier(max_depth=11, criterion='entropy')
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=11,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [9]:
# scores
print( dt.score(X_train, y_train) )
print( dt.score(X_test, y_test) )

0.999476850641
0.878686700056


In [34]:
np.set_printoptions(precision=4)
print("depth,estimator,stump/train,stump/score,stump_time,ada/train,ada/score,ada_time")
estimators = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
for depth in range (1,12):
    for estimator in estimators:
        #stump
        dt_stump = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=1)
        start_stump_time = time()
        dt_stump.fit(X_train, y_train)
        end_stump_time = time() - start_stump_time
        stump_score_train = dt_stump.score(X_train,y_train.values.ravel())
        stump_score_test = dt_stump.score(X_test, y_test)
        
        #ada boost
        ada = AdaBoostClassifier(base_estimator=dt_stump,n_estimators=estimator)
        start_ada_time = time()
        ada.fit(X_train, y_train.values.ravel())
        end_ada_time = time() - start_ada_time
        
        ada_score_train = ada.score(X_train, y_train)
        ada_score_test = ada.score(X_test, y_test)
        print(depth,",", estimator, "," , stump_score_train , "," , stump_score_test , "," , end_stump_time , ",",
              ada_score_train , "," , ada_score_test, "," , end_ada_time)

depth,estimator,stump/train,stump/score,stump_time,ada/train,ada/score,ada_time
1 , 50 , 0.193565262883 , 0.198107957707 , 0.008599042892456055 , 0.579911064609 , 0.565943238731 , 0.43619704246520996
1 , 100 , 0.193565262883 , 0.198107957707 , 0.005627870559692383 , 0.579911064609 , 0.565943238731 , 0.8389050960540771
1 , 150 , 0.193565262883 , 0.198107957707 , 0.0056040287017822266 , 0.579911064609 , 0.565943238731 , 1.1922581195831299
1 , 200 , 0.193565262883 , 0.198107957707 , 0.005722999572753906 , 0.579911064609 , 0.565943238731 , 1.743851900100708
1 , 250 , 0.193565262883 , 0.198107957707 , 0.005599021911621094 , 0.579911064609 , 0.565943238731 , 2.065117120742798
1 , 300 , 0.193565262883 , 0.198107957707 , 0.005776166915893555 , 0.579911064609 , 0.565943238731 , 2.454028844833374
1 , 350 , 0.193565262883 , 0.198107957707 , 0.005715131759643555 , 0.579911064609 , 0.565943238731 , 3.0422699451446533
1 , 400 , 0.193565262883 , 0.198107957707 , 0.00603795051574707 , 0.579911064609 ,

In [5]:
# cross validation
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

dt_stump = DecisionTreeClassifier(max_depth=6, min_samples_leaf=1)
ada = AdaBoostClassifier(base_estimator=dt_stump,n_estimators=50)
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
scores = cross_val_score(ada, X_train, y_train.values.ravel(), cv=cv)

print(scores.mean())


0.965998256321
